In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# 1. Data Loading and Preprocessing
# Define a transform to convert images to tensors.
transform = transforms.Compose([
    transforms.ToTensor(),
    # Optionally, add normalization:
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
# Download and load the CIFAR-10 training and test datasets.
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


100%|██████████| 170M/170M [00:11<00:00, 14.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

In [10]:
# 2. Define the CNN Model
# Here we subclass nn.Module to define a simple CNN with two convolutional layers,
# pooling, and two fully-connected layers.
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Convolutional layer 1: from 3 input channels to 32 output channels
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        # Activation (ReLU) and pooling follow
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Convolutional layer 2: from 32 channels to 64 channels
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        # A fully connected layer that takes the flattened features.
        # For CIFAR-10 images of size 32x32, after two poolings (each halving dimensions),
        # the spatial size becomes 8x8, and with 64 channels, that gives 64*8*8 = 4096 features.
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)  # 10 output classes

    def forward(self, x):
        # First conv layer + activation + pooling.
        x = self.pool(self.relu(self.conv1(x)))
        # Second conv layer + activation + pooling.
        x = self.pool(self.relu(self.conv2(x)))
        # Flatten the feature map to a vector
        x = x.view(x.size(0), -1)
        # Fully connected layers with activation
        x = self.relu(self.fc1(x))
        # Final fully connected layer (logits output)
        x = self.fc2(x)
        return x